In [1]:
import pandas as pd
import numpy as np
import json, time, xmltodict, os, urllib
from pathlib import Path

In [2]:
# task specific imports
from Bio import Entrez

In [3]:
base_dir = Path("../")
parsed_data_dir = base_dir / "parsed_data"
raw_data_dir = base_dir / "raw_data"

In [4]:
Entrez.email = "-"    

## Series level metadata

In [7]:
SERIES_ACCESSION = "GSE60141"

In [9]:
samples = []
series_data = {}

handle = Entrez.esearch(db="gds", term=f"{SERIES_ACCESSION}[Accession] AND gse[Filter]", field="acc", retmax=1)
series_result = Entrez.read(handle)
series_uid = series_result["IdList"][0]
print(SERIES_ACCESSION, series_uid)

series_data["esearch"] = series_result

# Get samples from the series
handle = Entrez.esummary(db="gds", id=series_uid, retmax=1000)
series_result = Entrez.read(handle)
series_samples = series_result[0]['Samples']
samples += series_samples
print(len(series_samples))

series_data["esummary"] = series_result

GSE60141 200060141
936


In [10]:
all_sample_details = {}
all_gsm_details = {}

## Sample level metadata
### Attempt to load previously collected data

In [15]:
if (raw_data_dir / "sample_metadata_dump.json").exists():
    with open(raw_data_dir / "sample_metadata_dump.json", "r") as f: all_sample_details = json.load(f)
else:
    all_sample_details = {}
print(len(all_sample_details))

942


In [14]:
if (raw_data_dir / "gsm_metadata_dump.json").exists():
    with open(raw_data_dir / "gsm_metadata_dump.json", "r") as f: all_gsm_details = json.load(f)  
else:
    all_gsm_details = {}
print(len(all_gsm_details))    

942


### Fetch anything missing

In [17]:
failed_samples = []
for s in samples:
    if not (s["Accession"] in all_gsm_details):
        try:
            # GEO dataset
            # Get GSM UID
            handle = Entrez.esearch(db="gds", term=f"{s['Accession']}[Accession] AND gsm[Filter]", field="acc", retmax=1)
            sample = Entrez.read(handle)
            sample_uid = sample["IdList"][0]
            # Get sample details
            handle = Entrez.esummary(db="gds", id=sample_uid, retmax=100)
            sample_details = Entrez.read(handle)[0]
            all_gsm_details[s["Accession"]] = sample_details
            time.sleep(1)
        except:
            print(s)
            failed_samples.append(s)
    if not (s["Accession"] in all_sample_details):
        try:
            # BioSample
            # Get sample UID        
            handle = Entrez.esearch(db="biosample", term=f"{s['Accession']}[Accession]", field="acc", retmax=1)
            sample = Entrez.read(handle)
            sample_uid = sample["IdList"][0]
    
            # Get sample details
            handle = Entrez.esummary(db="biosample", id=sample_uid, retmax=100)
            sample_details = Entrez.read(handle)
            
            all_sample_details[s["Accession"]] = sample_details
    
        except:
            print(s)
            failed_samples.append(s)


In [18]:
with open(raw_data_dir / "sample_metadata_dump.json", "w") as f: json.dump(all_sample_details, f)
with open(raw_data_dir / "gsm_metadata_dump.json", "w") as f: json.dump(all_gsm_details, f)

In [19]:
failed_samples

[]

## Prepare metadata table

In [20]:
keep_sample_details = {}

keep_attributes = [
    "gene id", 
    "protein affinity tag", 
    "expression system", 
    "dna source", 
    "protein", 
    "protein family", 
    "replicate", 
    "subset"
]

for acc, sample_details in all_sample_details.items():
    document_summary = sample_details['DocumentSummarySet']['DocumentSummary'][0]
    sample_data = xmltodict.parse(document_summary['SampleData'])
    attributes = sample_data['BioSample']['Attributes']['Attribute']
    for d in attributes:
        if d['@attribute_name'] in keep_attributes:
            all_gsm_details[acc][d['@attribute_name']] = d['#text']    


In [21]:
len(all_gsm_details)

942

In [22]:
df = pd.DataFrame.from_dict(all_gsm_details, orient="index")
df.index.name = "accession"
df = df[['title', 'summary', 'gene id', 'taxon', 'protein', 'protein family', 'protein affinity tag', 'expression system', 'dna source', 'replicate', 'subset']]
df.replace("", pd.NA, inplace=True)
print(df.shape)
df.head(3)

(942, 11)


,title,summary,gene id,taxon,protein,protein family,protein affinity tag,expression system,dna source,replicate,subset
accession,,,,,,,,,,,
GSM1466346,ChIPSeq-ABI5_mock_2,ChIPSeq-ABI5_mock,NaN,Arabidopsis thaliana,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM1466343,ChIPSeq-ABI5_Ypet_1,ChIPSeq-ABI5_Ypet,NaN,Arabidopsis thaliana,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM1466345,ChIPSeq-ABI5_Ypet_2,ChIPSeq-ABI5_Ypet,NaN,Arabidopsis thaliana,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filter table to samples with applicable DAPseq data

In [23]:
df = df[df["taxon"] == "Arabidopsis thaliana"]
df = df[df["subset"]=="motif+TFBS"]

In [24]:
df.groupby(["gene id", "dna source"]).size()

gene id    dna source
AT1G01060  col           1
           colamp        1
AT1G01250  col           1
AT1G01720  col           1
AT1G02230  col           1
                        ..
AT5G66940  colamp        1
AT5G67190  col           1
           colamp        1
AT5G67300  col           2
AT5G67580  col           1
Length: 568, dtype: int64

In [25]:
df["replicate"].value_counts()

replicate
a      503
b       38
v3a     35
v3b     26
v31     15
v3d      1
v3h      1
v35      1
v3e      1
Name: count, dtype: int64

Some samples (replicates) are merged, need to identify the "common" study title, and thus the common file. 

In [26]:
def get_common(x):
    # x = list(x)
    if len(x) == 1:
        return x.iloc[0]
    elif x.iloc[0] == x.iloc[1]:
        return x.iloc[0]
    else:
        print(f"{x.index.values}\t{x.iloc[0]}\t{x.iloc[1]}")
        return None

agg = {
    c: get_common for c in ['summary', 'taxon', 'protein', 'protein family', 'protein affinity tag', 'expression system']
}

agg['replicate'] = ','.join
agg['title'] = list
agg['accession'] = list
# agg['replicate'] = list


df = df.reset_index().groupby(["gene id", "dna source"]).agg(agg)

In [27]:
df['common title'] = df['title'].apply(lambda x: os.path.commonprefix(list(x)).rstrip("v3").rstrip("_"))

In [28]:
df.shape

(568, 10)

In [29]:
df["protein affinity tag"].value_counts()

protein affinity tag
Halo    566
GST       2
Name: count, dtype: int64

In [30]:
df["expression system"].value_counts()

expression system
tnt      566
ecoli      2
Name: count, dtype: int64

# Download files...

In [31]:
from ftplib import FTP
from datetime import datetime
import tarfile

In [33]:
ftp_link = series_data['esummary'][0]['FTPLink']; ftp_link

'ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE60nnn/GSE60141/'

In [34]:
url_parts = urllib.parse.urlparse(ftp_link)
ftp_base = url_parts.netloc
ftp_path = url_parts.path
ftp_base, ftp_path

('ftp.ncbi.nlm.nih.gov', '/geo/series/GSE60nnn/GSE60141/')

In [42]:
ftp = FTP(ftp_base)
ftp.login('','')
ftp.cwd(f'/{ftp_path}/suppl/')  

'250 CWD command successful'

In [43]:
start = datetime.now()

# Get All Files
files = ftp.nlst()

# Print out the files
for file in files:
    if (raw_data_dir / SERIES_ACCESSION / file).exists():
        print(file, "already downloaded.")
    else:
        print("Downloading..." + file)
        ftp.retrbinary("RETR " + file, open( raw_data_dir / SERIES_ACCESSION / file, 'wb').write)

ftp.quit()

end = datetime.now()
diff = end - start

GSE60141_DAPSeq-AP2EREBP_tnt-At1g22810_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-AP2EREBP_tnt-At2g33710_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-AP2EREBP_tnt-At4g32800_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-AP2EREBP_tnt-ERF105_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-AP2EREBP_tnt-ERF15_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-AP2EREBP_tnt-ERF6_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-ARF_ecoli-MP_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-C2H2_tnt-At5g66730_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-C2H2_tnt-IDD5_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-C2H2_tnt-STZ_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_DAPSeq-CPP_tnt-SOL1_col-chr1-5_GEM_events.narrowPeak.gz already downloaded.
GSE60141_D

In [44]:
for f in (raw_data_dir / SERIES_ACCESSION).glob('*'):
    if f.suffix == ".tar":
        tar = tarfile.open(f)
        tar.extractall(raw_data_dir / SERIES_ACCESSION)
        tar.close()

### File names
Add downloaded file names to metadata

In [45]:
def file_name(x):
    if len(x["accession"]) > 1:
        prefix = SERIES_ACCESSION
    else:
        prefix = x["accession"][0]
    
    file_name = raw_data_dir / SERIES_ACCESSION / f"{prefix}_DAPSeq-{x['common title'].replace('.', '-')}-chr1-5_GEM_events.narrowPeak.gz"

    if file_name.exists():
        return file_name
    else:
        print(file_name)
        return None
df["file_name"] = df.apply(file_name, axis=1)

In [46]:
df['title'] = df['title'].apply(lambda x: ','.join(sorted(x)))
df['accession'] = df['accession'].apply(lambda x: ','.join(sorted(x)))

## Save metadata

In [47]:
df.to_csv(parsed_data_dir / "sample_metadata.tsv", sep="\t")
parsed_data_dir / "sample_metadata.tsv"

PosixPath('../parsed_data/sample_metadata.tsv')

In [48]:
!head ../parsed_data/sample_metadata.tsv

gene id	dna source	summary	taxon	protein	protein family	protein affinity tag	expression system	replicate	title	accession	common title	file_name
AT1G01060	col	young leaf	Arabidopsis thaliana	LHY1	MYBrelated	Halo	tnt	a	MYBrelated_tnt.LHY1_col_a	GSM1925527	MYBrelated_tnt.LHY1_col_a	../raw_data/GSE60141/GSM1925527_DAPSeq-MYBrelated_tnt-LHY1_col_a-chr1-5_GEM_events.narrowPeak.gz
AT1G01060	colamp	young leaf	Arabidopsis thaliana	LHY1	MYBrelated	Halo	tnt	a	MYBrelated_tnt.LHY1_colamp_a	GSM1925526	MYBrelated_tnt.LHY1_colamp_a	../raw_data/GSE60141/GSM1925526_DAPSeq-MYBrelated_tnt-LHY1_colamp_a-chr1-5_GEM_events.narrowPeak.gz
AT1G01250	col	young leaf	Arabidopsis thaliana	AT1G01250	AP2EREBP	Halo	tnt	a	AP2EREBP_tnt.AT1G01250_col_a	GSM1925016	AP2EREBP_tnt.AT1G01250_col_a	../raw_data/GSE60141/GSM1925016_DAPSeq-AP2EREBP_tnt-AT1G01250_col_a-chr1-5_GEM_events.narrowPeak.gz
AT1G01720	col	young leaf	Arabidopsis thaliana	ATAF1	NAC	Halo	tnt	a	NAC_tnt.ATAF1_col_a	GSM1925578	NAC_tnt.ATAF1_col_a	../raw_data/GSE

In [123]:
# END

A check that the files match Ecker files... 

In [31]:
path = "../raw_data/not-used/dap_download_may2016_peaks.zip"
import os
import zipfile

ecker_file_titles = set()
for item in zipfile.ZipFile(path).namelist():
    ecker_file_titles.update([".".join(item.split("/")[2:4])])

In [32]:
len(ecker_file_titles)

568

In [33]:
df[df["common title"].isin(ecker_file_titles)].shape[0]

568

In [34]:
df[~df["common title"].isin(ecker_file_titles)]

,,summary,gene id,taxon,protein,protein family,protein affinity tag,expression system,dna source,title,replicate,common title
gene id,dna source,,,,,,,,,,,
